## ETL with Spark

#### Import Libraries

In [1]:
import os
import configparser
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

#### Create spark session w/ S3 config

In [ ]:
#config = configparser.ConfigParser()

#Normally this file should be in ~/.aws/credentials
#config.read_file(open('dl.cfg'))

#os.environ["AWS_ACCESS_KEY_ID"]= config['AWS']['AWS_ACCESS_KEY_ID']
#os.environ["AWS_SECRET_ACCESS_KEY"]= config['AWS']['AWS_SECRET_ACCESS_KEY']

os.environ["AWS_ACCESS_KEY_ID"] = ###
os.environ["AWS_SECRET_ACCESS_KEY"] = ###

In [2]:
spark = SparkSession\
        .builder\
        .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
        .getOrCreate()

In [4]:
# Update to ne s3 bucket url
output_data = 's3a://patrickudacity-songplays/'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Load data from S3 instead

In [3]:
song_df = spark.read.json('s3a://udacity-dend/song_data/*/*/*/*.json')

Py4JJavaError: An error occurred while calling o32.json.
: com.amazonaws.AmazonClientException: Unable to load AWS credentials from any provider in the chain
	at com.amazonaws.auth.AWSCredentialsProviderChain.getCredentials(AWSCredentialsProviderChain.java:117)
	at com.amazonaws.services.s3.AmazonS3Client.invoke(AmazonS3Client.java:3521)
	at com.amazonaws.services.s3.AmazonS3Client.headBucket(AmazonS3Client.java:1031)
	at com.amazonaws.services.s3.AmazonS3Client.doesBucketExist(AmazonS3Client.java:994)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:297)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2669)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at org.

In [32]:
song_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14896

In [8]:
song_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

#### Creat songs table

In [10]:
song_df.createOrReplaceTempView("songdata")

# song_id, title, artist_id, year, duration
songs_table = spark.sql("""
    SELECT song_id, title, artist_id, year, duration
    FROM songdata
""")

songs_table.limit(3).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOSIJKW12A8C1330E3|A God In An Alcov...|ARKIQCZ1187B9A7C7C|   0|248.65914|
|SODVVEL12A6D4F9EA0|Good Old Wagon (L...|ARI9DQS1187B995625|1964|139.78077|
|SOKPKMV12A8C14125E|Catwalk (Black In...|AR2L9A61187B9ADDBC|1995|492.30322|
+------------------+--------------------+------------------+----+---------+

#### Create artists table

In [11]:
# artist_id, name, location, lattitude, longitude
artists_table = spark.sql("""
    SELECT DISTINCT artist_id, artist_name, artist_location, artist_latitude, artist_longitude
    FROM songdata
""")

artists_table.limit(3).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-------------+------------------+---------------+----------------+
|         artist_id|  artist_name|   artist_location|artist_latitude|artist_longitude|
+------------------+-------------+------------------+---------------+----------------+
|AR35RMH1187FB4A865|Eumir Deodato|    Rio de Janeiro|      -22.97673|       -43.19508|
|ARN4G4X1187FB485B4|  The Donkeys|Great Britain / UK|       54.31392|        -2.23218|
|ARQTC851187B9B03AF|       O.A.R.|     Rockville, MD|       39.08166|       -77.15124|
+------------------+-------------+------------------+---------------+----------------+

#### Write artists table and song table to parquet file

In [12]:
songs_table.write.partitionBy('year', 'artist_id').parquet(os.path.join(output_data, 'songs_table'), 'overwrite')
artists_table.write.parquet(os.path.join(output_data, 'artists_table'), 'overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Load log data to dataframe

In [33]:
# Limiting to Nov 2018 data
log_df = spark.read.json('s3a://udacity-dend/log_data/*/*/*.json')
log_df.limit(3).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+
|     artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|           song|status|           ts|           userAgent|userId|
+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+
|   Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|  Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|
|The Prodigy|Logged In|     Ryan|     M|            1|   Smith|260.07465| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|The Big Gundown|

In [34]:
log_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8056

In [19]:
log_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [20]:
# user_id, first_name, last_name, gender, level
log_df.createOrReplaceTempView("logdata")

users_table = spark.sql("""
    SELECT DISTINCT userId, firstName, lastName, gender, level
    FROM logdata
    WHERE page = 'NextSong'
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
#### Write to parquet file

In [21]:
users_table.write.parquet(os.path.join(output_data, 'users_table'), 'overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Create time table. Convert ts to time_stamp

In [35]:
# start_time, hour, day, week, month, year, weekday
log_df = log_df.withColumn('ts', F.to_timestamp(F.col('ts')/1000))

log_df.createOrReplaceTempView("logdata")

time_table = spark.sql("""
    SELECT DISTINCT ts as start_time, hour(ts) as hour, day(ts) as day, month(ts) as month, year(ts) as year, dayofweek(ts) as weekday
    FROM logdata
""")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
time_table.limit(3).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+---+-----+----+-------+
|          start_time|hour|day|month|year|weekday|
+--------------------+----+---+-----+----+-------+
|2018-11-12 14:20:...|  14| 12|   11|2018|      2|
|2018-11-12 17:55:...|  17| 12|   11|2018|      2|
|2018-11-12 08:58:...|   8| 12|   11|2018|      2|
+--------------------+----+---+-----+----+-------+

#### Write time table to parquet file. Partition by year / month

In [24]:
time_table.write.partitionBy('year', 'month').parquet(os.path.join(output_data, 'time_table'), 'overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Creat Songplay table

In [ ]:
# songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent
songplay_table = spark.sql("""
    SELECT l.ts as start_time, year(l.ts) as year, month(l.ts) as month,
           l.userId, l.level, s.song_id, s.artist_id, 
           l.sessionId, l.location, l.userAgent
    FROM logdata l
    LEFT JOIN songdata s
    ON l.artist = s.artist_name AND l.song = s.title
""")
songplay_table.limit(3).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
songplay_table.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

213

#### Add song_play id

In [28]:
songplay_table = songplay_table.withColumn('songplay_id' , F.monotonically_increasing_id())
songplay_table.limit(2).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+-----+------+-----+-------+---------+---------+--------------------+--------------------+-----------+
|          start_time|year|month|userId|level|song_id|artist_id|sessionId|            location|           userAgent|songplay_id|
+--------------------+----+-----+------+-----+-------+---------+---------+--------------------+--------------------+-----------+
|2018-11-12 02:36:...|2018|   11|    53| free|   null|     null|      438|   Klamath Falls, OR|"Mozilla/5.0 (Win...|          0|
|2018-11-12 02:37:...|2018|   11|    10| free|   null|     null|      345|Washington-Arling...|"Mozilla/5.0 (Mac...|          1|
+--------------------+----+-----+------+-----+-------+---------+---------+--------------------+--------------------+-----------+

#### Write time table to parquet file. Partition by year / month

In [29]:
songplay_table.write.partitionBy('year', 'month').parquet(os.path.join(output_data, 'songplay_table'), 'overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…